## Note allocator ver3.0
Key difference: consider about time & tempo

### Constraints
1. num of pianists, 2 hands, 5 fingers
2. spatial apart (in each hand cannot exceed one octave)
3. within one octave > same intensity > last for same period
4. hand shape

In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from midiutil import MIDIFile
from pprint import pprint

from utils import *

import copy
import random

In [ ]:
## Parameters
group_num = 21 * 2 #21 pianists, 2 hands each
max_note_per_group = 5 #5 fingers

#min_pitch_gap = not set
win_length = 25e-3
min_time_gap = 0.3
min_frame_gap = int(round(min_time_gap / win_length))

In [ ]:
def scale_int(intensity_map):
    #scale the intensity map(0-1) to maximum=scale
    scale = 36
    l = np.array(intensity_map)
    l = l * scale / np.amax(l)
    return l.tolist()

def intensity2sign(intensity):
    #quantize the intensity -> to dynamic sign
    int_nm = round(intensity/16)*16
    if int_nm <=16:
        return 'ppp'
    elif int_nm ==32:
        return 'pp'
    elif int_nm ==48:
        return 'p'
    elif int_nm ==64:
        return 'mp'
    elif int_nm ==80:
        return 'mf'
    elif int_nm ==96:
        return 'f'
    elif int_nm ==112:
        return 'ff'
    elif int_nm >=127:
        return 'fff'
    
def int2vel(intensity):
    #intensity(in dB) to loudness, lowest loudness is (0dB) 16  
    base = 16
    return 10**(intensity/40) * base

def picknotes(cur_key, cur_freq, cur_int, notes, notes_list):
    j = 0
    while j < len(cur_key):
        if cur_key[j] == 0:
            j += 1
            continue
        pitch = int(cur_key[j])
        while pitch > 88:
            pitch -= 12
        
        start = j
        ori_int = cur_int[j]
        while cur_key[j] != 0 and j < len(cur_key)-1:
            j += 1
            ori_int = max(ori_int,cur_int[j])
        end = j - 1
        if end - start < 3: 
            end = start + 3 #if 2, all to 75ms; if 3, all to 100ms.
     
        intensity = get_intensity(cur_key[j-1],cur_freq,notes_list) * ori_int
        #intensity = ori_int
        intensity = round(int2vel(intensity),3)
        #intensity = get_intensity(cur_key[j-1],cur_freq,notes_list) * 100
        intensity_sign = intensity2sign(intensity)
        notes.append(Note(pitch, start, end, intensity, intensity_sign))
        j += 1
    return notes

def remove_repetitive(notes,note_num):
    #changed
    note_set = [0]*note_num
    i = 0
    while i < len(notes):
        note = notes[i]
        if note_set[note.pitch-1] == 0:
            note_set[note.pitch-1] = [note]
        else:
            delete = False
            for n in note_set[note.pitch-1]:
                if n.start == note.start and n.end == note.end:
                    n.intensity = max(n.intensity, note.intensity)
                    n.intensity_sign = intensity2sign(n.intensity)
                    del notes[i]
                    delete = True
                    i -= 1
                    break
            if not delete:
                note_set[note.pitch-1].append(note)
        i += 1
    return notes

In [ ]:
key_time = 50e-3*0.5
Fs = 48000

## picked keys
#keys = scipy.io.loadmat('MATLAB_data/23Feb_jc_key_2octave.mat')['key']
keys = scipy.io.loadmat('MATLAB_data/23Feb_jc_key_2octave.mat')['key']

##freq of notes on piano
notes_list = scipy.io.loadmat('notes_112.mat')['notes'] 

##freq of t-f components
freq_list = scipy.io.loadmat('freq_480_two_octave_down.mat')['f_cropped'] 

intensity_map = scipy.io.loadmat('MATLAB_data/23Feb_jc_int_2octave.mat')['filtered_int_db']
intensity_map = scale_int(intensity_map)

notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

note_num = 112

notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_int = intensity_map[i]
    cur_freq = freq_list[i]
    notes = picknotes(cur_key, cur_freq, cur_int, notes, notes_list)

notes = sorted(notes, key = lambda note: note.start)

new_notes = remove_repetitive(notes,note_num)
#pprint(new_notes)
separate_majors(new_notes)


In [ ]:
#pprint(new_notes)
print(len(new_notes))
plot_notes(new_notes)

In [ ]:
word_gap = 15
def separate_word(notes,word_gap):
    word_list = []
    ptr = 0
    for i in range(len(notes)-1):
        if notes[i+1].start - notes[i].end >= word_gap:
            word_list.append(notes[ptr:i+1])
            ptr = i + 1
        if i == len(notes) - 2:
            word_list.append(notes[ptr:])
    return word_list
    

In [ ]:
def check_handshape(pitchs,righthand):
    n = len(pitchs) #number of notes in the group
    pitchs.sort()
    constraints = {'12': 12, '13': 12, '14': 12, '15': 12, '23': 6, '24': 7, '25': 10, '34': 4, '35': 6, '45': 4}
    if n < 3:
        return True
    if righthand:
        #right hand
        if n == 3:
            if pitchs[-1]-pitchs[-2] > 9: return False
            return True
        if n == 4:
            g1 = pitchs[1] - pitchs[0] 
            g2 = pitchs[2] - pitchs[1] 
            g3 = pitchs[3] - pitchs[2] 
            g4 = pitchs[2] - pitchs[0]
            g5 = pitchs[3] - pitchs[1]
            g6 = pitchs[3] - pitchs[0]
            #1234
            if g1 <= constraints['12'] and g2 <= constraints['23'] and g3 <= constraints['34'] and g4 <= constraints['13'] and g5 <= constraints['24'] and g6 <= constraints['14']:
                return True
            #1235
            elif g1 <= constraints['12'] and g2 <= constraints['23'] and g3 <= constraints['35'] and g4 <= constraints['13'] and g5 <= constraints['25'] and g6 <= constraints['15']:
                return True
            #1245
            elif g1 <= constraints['12'] and g2 <= constraints['24'] and g3 <= constraints['45'] and g4 <= constraints['14'] and g5 <= constraints['25'] and g6 <= constraints['15']:
                return True
            #1345
            elif g1 <= constraints['13'] and g2 <= constraints['34'] and g3 <= constraints['45'] and g4 <= constraints['14'] and g5 <= constraints['35'] and g6 <= constraints['15']:
                return True
            #2345
            elif g1 <= constraints['23'] and g2 <= constraints['34'] and g3 <= constraints['45'] and g4 <= constraints['24'] and g5 <= constraints['35'] and g6 <= constraints['25']:
                return True
            return False
        if n == 5:
            g1 = (pitchs[1] - pitchs[0]) <= constraints['12']
            g2 = (pitchs[2] - pitchs[0]) <= constraints['13']
            g3 = (pitchs[3] - pitchs[0]) <= constraints['14']
            g4 = (pitchs[4] - pitchs[0]) <= constraints['15']
            g5 = (pitchs[2] - pitchs[1]) <= constraints['23']
            g6 = (pitchs[3] - pitchs[1]) <= constraints['24']
            g7 = (pitchs[4] - pitchs[1]) <= constraints['25']
            g8 = (pitchs[3] - pitchs[2]) <= constraints['34']
            g9 = (pitchs[4] - pitchs[2]) <= constraints['35']
            g10 = (pitchs[4] - pitchs[3]) <= constraints['45']
            return all([g1, g2, g3, g4, g5, g6, g7, g8, g9, g10])
    else:
        #left hand
        if n == 3:
            if pitchs[1]-pitchs[0] > 9: return False
            return True
        if n == 4:
            g1 = pitchs[1] - pitchs[0] 
            g2 = pitchs[2] - pitchs[1] 
            g3 = pitchs[3] - pitchs[2] 
            g4 = pitchs[2] - pitchs[0]
            g5 = pitchs[3] - pitchs[1]
            g6 = pitchs[3] - pitchs[0]
            #4321
            if g1 <= constraints['34'] and g2 <= constraints['23'] and g3 <= constraints['12'] and g4 <= constraints['24'] and g5 <= constraints['13'] and g6 <= constraints['14']:
                return True
            #5321
            elif g1 <= constraints['35'] and g2 <= constraints['23'] and g3 <= constraints['12'] and g4 <= constraints['25'] and g5 <= constraints['13'] and g6 <= constraints['15']:
                return True
            #5421
            elif g1 <= constraints['45'] and g2 <= constraints['24'] and g3 <= constraints['12'] and g4 <= constraints['25'] and g5 <= constraints['14'] and g6 <= constraints['15']:
                return True
            #5431
            elif g1 <= constraints['45'] and g2 <= constraints['34'] and g3 <= constraints['13'] and g4 <= constraints['35'] and g5 <= constraints['14'] and g6 <= constraints['15']:
                return True
            #5432
            elif g1 <= constraints['45'] and g2 <= constraints['34'] and g3 <= constraints['23'] and g4 <= constraints['35'] and g5 <= constraints['24'] and g6 <= constraints['25']:
                return True
            return False
        if n == 5:
            g1 = (pitchs[1] - pitchs[0]) <= constraints['45']
            g2 = (pitchs[2] - pitchs[0]) <= constraints['35']
            g3 = (pitchs[3] - pitchs[0]) <= constraints['25']
            g4 = (pitchs[4] - pitchs[0]) <= constraints['15']
            g5 = (pitchs[2] - pitchs[1]) <= constraints['34']
            g6 = (pitchs[3] - pitchs[1]) <= constraints['24']
            g7 = (pitchs[4] - pitchs[1]) <= constraints['14']
            g8 = (pitchs[3] - pitchs[2]) <= constraints['23']
            g9 = (pitchs[4] - pitchs[2]) <= constraints['13']
            g10 = (pitchs[4] - pitchs[3]) <= constraints['12']
            return all([g1, g2, g3, g4, g5, g6, g7, g8, g9, g10])
    print('Group not valid')
    return False

In [ ]:
def valid_note_to_allocate(note, righthand, pitch_group, high, low, newgroup,test = False):
    test_group = pitch_group + [note.pitch]
    if not check_handshape(test_group,righthand):
        return False
    base = note.pitch if pitch_group == [] else min(pitch_group)
    top = note.pitch if pitch_group == [] else max(pitch_group)
    
    if (note.pitch not in pitch_group) and ((top - 12) <= note.pitch <= (12 + base)):
        #q remain: limit highest note range?
        if newgroup:
            return True
        if righthand and note.pitch <= high:
            return False
        if not righthand and note.pitch >= low:
            return False
        return True
    return False

def valid_to_add(candidate,current_group):
    # numel = sum([len(x) for x in candidate])
    # c1 = counter < numel
    c1 = (not empty_list(candidate))
    c2 = len(current_group) < 5
    return c1 and c2

In [ ]:
def priority(note, current_allocated_note,major = 'None'):
    c1 = note.intensity_sign == current_allocated_note.intensity_sign
    c2 = (note.start == current_allocated_note.start and note.end == current_allocated_note.end)
    # c3 =  note.major == major

    if c1 and c2:
        return 1
    elif c1 and (not c2):
        return 2
    elif c2 and (not c1):
        return 3
    else: return 4


In [ ]:
class Pianist:
    def __init__(self, num, starttime = None, tempo = None):
        self.num = num + 1
        self.leftnotes = []
        self.rightnotes = []
        self.starttime = starttime
        self.tempo = tempo
        self.lendtime = 0 
        self.rendtime = 0
        self.high = 0 #highest note of left hand
        self.low = 89 #lowest note of right hand

    def __repr__(self):
        return f"Pianist No.{self.num}. Tempo = {self.tempo}bpm, starting time = {self.starttime}"
    
    def addleft(self,notes):
        self.leftnotes.extend(notes)
        end = max([note.end for note in notes])
        self.lendtime = max(self.lendtime, end)

    def addright(self,notes):
        self.rightnotes.extend(notes)
        end = max([note.end for note in notes])
        self.rendtime = max(self.rendtime, end)

    def endtimes(self):
        print('Pianist No.', self.num, 'left ending time =', self.lendtime, 'right ending time =', self.rendtime)
        
    def printnotes(self):
        print('Pianist No.', self.num)
        print('Left hand')
        pprint(self.leftnotes)
        print('Right hand')
        pprint(self.rightnotes)

In [ ]:
def init_pianists():
    pianists = []
    for i in range(21):
        pianists.append(Pianist(i))
    return pianists

In [ ]:
def allocation(cur_pianist, righthand, valid_notes_group, notes_group, newgroup, tempo = None):
    current_group = []
    pitch_group = []
    
    if newgroup:
        high = 0
        low = 89
        if not righthand:
            minnote = min(valid_notes_group, key = lambda note: note.pitch)
            pitch_group = [minnote.pitch]
            #major = minnote.major
            current_group.append(minnote)
            valid_notes_group.remove(minnote)
        else:
            maxnote = max(valid_notes_group, key = lambda note: note.pitch)
            pitch_group = [maxnote.pitch]
            #major = minnote.major
            current_group.append(maxnote)
            valid_notes_group.remove(maxnote)
    else:
        if not righthand:
            high = 0 #no lower limit
            low = cur_pianist.low #lowest of righthand, upper limit
            for i in range(len(valid_notes_group)):
                if valid_notes_group[i].pitch <= low:
                    pitch_group = [valid_notes_group[i].pitch]
                    current_group.append(valid_notes_group.pop(i))
                    break
        else:
            low = 89 #no upper limit
            high = cur_pianist.high #highest of lefthand, lower limit
            for i in range(len(valid_notes_group)):
                if valid_notes_group[i].pitch >= high:
                    pitch_group = [valid_notes_group[i].pitch]
                    current_group.append(valid_notes_group.pop(i))
                    break

    candidate = [[],[],[],[]]   #4 priority groups for candidates         
    for current_allocated_note in current_group:
        if len(current_group) >= 5:
            break
        candidate = [[],[],[],[]]
        for note in valid_notes_group:
            if not valid_note_to_allocate(note, righthand, pitch_group, high, low, newgroup):
                continue
            p = priority(note,current_allocated_note)
            candidate[p-1].append(note)

        for i in range(2): 
            #only consider the top two priority groups here
            #still notes in candidate list, current_group has not been filled up
            while valid_to_add(candidate[i],current_group):
                cur_candidates = candidate[i]
                shuffle_note = cur_candidates[random.randint(0, len(cur_candidates)-1)]
                candidate[i].remove(shuffle_note)
                if shuffle_note.pitch not in pitch_group:
                    if len(current_group) >= 2:
                        test_group = pitch_group + [shuffle_note.pitch]
                        if not check_handshape(test_group,righthand):
                            continue
                    pitch_group.append(shuffle_note.pitch)
                    current_group.append(shuffle_note)
                    valid_notes_group.remove(shuffle_note)


    for i in range(len(candidate)): 
        while valid_to_add(candidate[i],current_group):
            cur_candidates = candidate[i]
            shuffle_note = cur_candidates[random.randint(0, len(cur_candidates)-1)]
            candidate[i].remove(shuffle_note)
            if shuffle_note.pitch not in pitch_group:
                if len(current_group) >= 2:
                    test_group = pitch_group + [shuffle_note.pitch]
                    if not check_handshape(test_group,righthand):
                        continue
                pitch_group.append(shuffle_note.pitch)
                current_group.append(shuffle_note)
                valid_notes_group.remove(shuffle_note)
    

                
    

    if current_group == []:
        return notes_group, True
        

    

    if righthand:
        low = min([n.pitch for n in current_group])
        cur_pianist.low = low
        cur_pianist.addright(current_group)
    else:
        high = max([n.pitch for n in current_group])
        cur_pianist.high = high
        if cur_pianist.starttime == None:
            cur_pianist.starttime = min([n.start for n in current_group])
        cur_pianist.addleft(current_group)

    for note in current_group:
        notes_group.remove(note)


    return notes_group, False

In [ ]:
def current_pianist(pianists):
    num = 0
    righthand = False
    endtime = pianists[0].lendtime
    for idx, pianist in enumerate(pianists):
        if pianist.lendtime == 0:
            return pianist, False
        elif pianist.rendtime == 0:
            return pianist, True
        
        if pianist.lendtime <= endtime:
            endtime = pianist.lendtime
            righthand = False
            num = idx
        elif pianist.rendtime <= endtime:
            endtime = pianist.rendtime
            righthand = True
            num = idx
    
    return pianists[num], righthand

In [ ]:
def init_time_stack(pianists):
    time_stack = []
    for pianist in pianists:
        time_stack.append([pianist, False, pianist.lendtime])
        time_stack.append([pianist, True, pianist.rendtime])
    return time_stack

In [ ]:
def allocate_pianist(cur_pianist, notes_group):
    while notes_group != []:
        # print('pianist', cur_pianist.num)
        if cur_pianist.lendtime <= cur_pianist.rendtime:
            righthand = False 
            end_time = cur_pianist.lendtime
        else:
            righthand = True
            end_time = cur_pianist.rendtime
        if end_time == 0:
            start_time = 0
        else:
            start_time = end_time + min_frame_gap

        valid_notes_group = [note for note in notes_group if note.start >= start_time]
        if valid_notes_group == []:
            break

        start_frame = min([note.start for note in valid_notes_group])
        newgroup = True if start_frame >= max(cur_pianist.lendtime, cur_pianist.rendtime) else False

        end_frame = start_frame + 12 #earliest possible starting time + some gap = lastest starting time for the current considered group
        current_considered_notes = [note for note in valid_notes_group if note.start <= end_frame]

        
        notes_group, stop = allocation(cur_pianist, righthand, current_considered_notes, notes_group, newgroup)

        if stop:
            if righthand:
                cur_pianist.rendtime = end_frame
            else:
                cur_pianist.lendtime = end_frame

    return notes_group

In [ ]:
def allocation_all(notes_group,pianists):
    group = 0
    while notes_group != [] and group < 21:
        cur_pianist = pianists[group]
        notes_group = allocate_pianist(cur_pianist, notes_group)

        group += 1

    if notes_group != []:
        print('Failed :(    ',len(notes_group), 'notes unallocated.')
        return False
    else:
        print('Success!')
        return True
    

In [ ]:
word_groups = separate_word(new_notes,word_gap)
for item in word_groups:
    print(len(item))

In [ ]:
test_group = word_groups[1] + word_groups[2] + word_groups[3] + word_groups[4]

In [ ]:
pianists = init_pianists()
current_allocate_group = copy.deepcopy(new_notes)
result = allocation_all(current_allocate_group, pianists)
for pianist in pianists:
    pianist.printnotes()
pprint(current_allocate_group)

In [ ]:
pianists = init_pianists()
current_allocate_group = copy.deepcopy(new_notes)
result = False
while not result:
    pianists = init_pianists()
    result = allocation_all(current_allocate_group, pianists)
    current_allocate_group = copy.deepcopy(new_notes)
    random.shuffle(current_allocate_group)
for pianist in pianists:
    pianist.printnotes()
    

In [ ]:
result_all = []
for i in range(2,5):
    print(i)
    current_allocate_group = copy.deepcopy(word_groups[i])
    #random.shuffle(current_allocate_group)
    result = False
    while not result:
        result = allocation_all(current_allocate_group)
        current_allocate_group = copy.deepcopy(word_groups[i])
        random.shuffle(current_allocate_group)
    result_all.append(result)
   

In [ ]:
current_allocate_group = copy.deepcopy(word_groups[4])
result = False
while not result:
    result = allocation(current_allocate_group)
    current_allocate_group = copy.deepcopy(word_groups[4])
    random.shuffle(current_allocate_group)


In [ ]:
plot_notes(new_notes)
plt.show()
word_groups = separate_word(new_notes)
for g in word_groups:
    plot_notes(g)
plt.show()

In [ ]:
mf = MIDIFile(2)     # track number
time = 0    # start at the beginning
mf.addTrackName(0, time, "Right")
mf.addTrackName(1, time, "Left")
mf.addTempo(0, time, 1200)
mf.addTempo(1, time, 1200)

In [ ]:
for i in range(16):
    p = pianists[i]
    for note in p.rightnotes:    
        v = int(sign2vel(note.intensity_sign))
        mf.addNote(0,i,note.pitch+20, note.start, note.end-note.start+1,v)

    for note in p.leftnotes:    
        v = int(sign2vel(note.intensity_sign))
        mf.addNote(1,i,note.pitch+20, note.start, note.end-note.start+1,v)

In [ ]:
with open("midifile/8Mar_allocation_6.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
import numpy as np
array = np.array(result_all,dtype=object)
np.save("allocation_result_1Mar.npy", array)

In [ ]:
def allocation_all_old(notes_group):
    allocate_result=[]
    group = 1
    righthand = False #true if last group was left hand, current group is right hand, they should not overlap
    while notes_group != [] and group <= group_num:
        ## each while iteration allocate one group
        current_group = []
        pitch_group = []
        
        if not righthand:
            high = 0
            minnote = min(notes_group, key = lambda note: note.pitch)
            pitch_group = [minnote.pitch]
            major = minnote.major
            current_group.append(minnote)
            notes_group.remove(minnote)
        else:
            for i in range(len(notes_group)):
                if notes_group[i].pitch > (high + 1):
                    pitch_group = [notes_group[i].pitch]
                    current_group.append(notes_group.pop(i))
                    break
        candidate = [[],[],[],[]]   #4 priority groups for candidates         
        for current_allocated_note in current_group:
            if len(current_group) >= 5:
                break
            candidate = [[],[],[],[]]
            for note in notes_group:
                if not valid_note_to_allocate(note,righthand,pitch_group,high):
                    continue
                p = priority(note,current_allocated_note,major)
                candidate[p-1].append(note)


            for i in range(2): 
                #still notes in candidate list, current_group has not been filled up
                while valid_to_add(candidate[i],current_group):
                    cur_candidates = candidate[i]
                    shuffle_note = cur_candidates[random.randint(0, len(cur_candidates)-1)]
                    candidate[i].remove(shuffle_note)
                    if shuffle_note.pitch not in pitch_group:
                        if len(current_group) >= 2:
                            test_group = pitch_group + [shuffle_note.pitch]
                            if not check_handshape(test_group,righthand):
                                continue
                        pitch_group.append(shuffle_note.pitch)
                        current_group.append(shuffle_note)
                        notes_group.remove(shuffle_note)

        for i in range(len(candidate)): 
            while valid_to_add(candidate[i],current_group):
                cur_candidates = candidate[i]
                shuffle_note = cur_candidates[random.randint(0, len(cur_candidates)-1)]
                candidate[i].remove(shuffle_note)
                if shuffle_note.pitch not in pitch_group:
                    if len(current_group) >= 2:
                        test_group = pitch_group + [shuffle_note.pitch]
                        if not check_handshape(test_group,righthand):
                            continue
                    pitch_group.append(shuffle_note.pitch)
                    current_group.append(shuffle_note)
                    notes_group.remove(shuffle_note)


        if not righthand:
            high = max([n.pitch for n in current_group])

        allocate_result.append(current_group)

        righthand = not righthand
        group += 1 #finish allocate notes to one group, update
    
    counter = 0
    while counter < len(notes_group):
        note = notes_group[counter]
        for i, allocated_group in enumerate(allocate_result):
            if len(allocated_group) >= 4:
                continue
            if len(allocated_group) < 4:
                pitch_group = [n.pitch for n in allocated_group]
                high = max([n.pitch for n in allocate_result[i-1]]) if i % 2 == 0 else 0
                righthand = True if i % 2 == 0 else False
                if valid_note_to_allocate(note,righthand,pitch_group,high):
                    allocated_group.append(note)
                    notes_group.remove(note)
                    break
        counter += 1

    if notes_group != []:
        print('Failed :(    ',len(notes_group), 'notes unallocated.')
        return False
    else:
        print('Success!')
        return allocate_result

In [ ]:
note1 = Note(34,0,4,0)
note2 = Note(33,0,5,0)
note3 = Note(33,0,4,0)
note = [note1, note2, note3]
minnote = min(note, key = lambda note: note.pitch)
print(minnote)